In [1]:
from pyspark.sql import SparkSession

In [4]:
from pyspark.ml.regression import LinearRegression

In [5]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [9]:
data = spark.read.csv('Ecommerce_Customers.csv',inferSchema=True,
                     header=True)

In [10]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [15]:
for item in data.head(2)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [16]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [17]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [22]:
assembler = VectorAssembler(inputCols=['Avg Session Length', 'Time on App', 'Time on Website', 'Length of Membership', 'Yearly Amount Spent'],
                           outputCol = 'features')

In [23]:
output = assembler.transform(data)

In [24]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826, 587.9511]))]

In [26]:
final_data = output.select('features','Yearly Amount Spent')

In [28]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [60]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [61]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                353|
|   mean| 497.30493559941004|
| stddev|  78.12312579850749|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [62]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [63]:
lr_model = lr.fit(train_data)

In [64]:
test_results = lr_model.evaluate(test_data)

In [65]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|1.375610736431554...|
|7.275957614183426...|
|1.284661266254261...|
|-4.54747350886464...|
|6.821210263296962...|
|3.865352482534945...|
|7.787548383930698...|
|1.693933882052079...|
|2.842170943040400...|
|-1.42108547152020...|
|2.501110429875552...|
|-2.16004991671070...|
|1.233502189279534...|
|5.343281372915953...|
|4.092726157978177...|
|2.785327524179592...|
|8.924416761146858...|
|-4.71800376544706...|
|4.774847184307873...|
|2.557953848736360...|
+--------------------+
only showing top 20 rows



In [66]:
test_results.rootMeanSquaredError

6.270044300289485e-12

In [67]:
test_results.r2

1.0

In [68]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [69]:
unlabeled_data = test_data.select('features')

In [70]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[29.5324289670579...|
|[30.5743636841713...|
|[30.7377203726281...|
|[30.8162006488763...|
|[30.8364326747734...|
|[30.8794843441274...|
|[31.2681042107507...|
|[31.2834474760581...|
|[31.3895854806643...|
|[31.5171218025062...|
|[31.5257524169682...|
|[31.5702008293202...|
|[31.6005122003032...|
|[31.6610498227460...|
|[31.7216523605090...|
|[31.7242025238451...|
|[31.8124825597242...|
|[31.8164283341993...|
|[31.8293464559211...|
|[31.8745516945853...|
+--------------------+
only showing top 20 rows



In [71]:
predictions = lr_model.transform(unlabeled_data)

In [72]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...| 408.6403510726137|
|[30.5743636841713...| 442.0644137580584|
|[30.7377203726281...|461.78074219621703|
|[30.8162006488763...|266.08634094846946|
|[30.8364326747734...| 467.5019004269828|
|[30.8794843441274...| 490.2065999848508|
|[31.2681042107507...|423.47053317381614|
|[31.2834474760581...| 591.7810894256505|
|[31.3895854806643...|410.06961105998005|
|[31.5171218025062...|275.91842065038713|
|[31.5257524169682...| 443.9656268098794|
|[31.5702008293202...|  545.945492141407|
|[31.6005122003032...| 479.1728514910846|
|[31.6610498227460...| 416.3583535798955|
|[31.7216523605090...|347.77692663186855|
|[31.7242025238451...| 503.3878872879577|
|[31.8124825597242...| 392.8103449837883|
|[31.8164283341993...| 501.1224915036611|
|[31.8293464559211...| 385.1523379879702|
|[31.8745516945853...|392.28524424626494|
+--------------------+------------